In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pycolmap

from hloc import extract_features, pairs_from_covisibility, match_features, triangulation, pairs_from_retrieval, localize_sfm, visualization
from hloc.utils import viz_3d, io
from hloc.localize_sfm import QueryLocalizer, pose_from_cluster

In [ ]:
dataset_name = 'LabFront'

dataset = Path(f'datasets/{dataset_name}/')
image_dir = dataset / 'images'
colmap_model_path = dataset / 'colmap/sparse/0'

output_dir = dataset / 'hloc_data/'
# Local features and global descriptor filenames hardocded in feature confs
sfm_pairs_path = output_dir / 'sfm-pairs-covis20.txt' # Pairs used for SfM reconstruction
sfm_reconstruction_path = output_dir / 'sfm_reconstruction' # Path to reconstructed SfM

# Query images
queries_img_dir = dataset / 'query'

# Query data
query_processing_data_dir = Path(f'query_processing_data/{dataset_name}')
query_global_matches_path = query_processing_data_dir / 'global_match_pairs.txt'
query_local_match_path = query_processing_data_dir / 'local_match_data.h5'
query_results = query_processing_data_dir / 'query_results.txt'

In [ ]:
# Feature extraction
## Extract local features in each data set image using Superpoint
local_feature_conf = extract_features.confs['superpoint_aachen']
local_features_path = extract_features.main(
    conf = local_feature_conf,
    image_dir = image_dir,
    export_dir = output_dir
)

## Extract global descriptors from each image using NetVLad
global_descriptor_conf = extract_features.confs['netvlad']
global_descriptors_path = extract_features.main(
    conf = global_descriptor_conf,
    image_dir = image_dir,
    export_dir = output_dir
)

In [ ]:
# Create SfM model using the local features just extracted

## Note: There is already an SfM model created using Colmap available. However, that is created using the RootSIFT features.
## SfM model needs to be created using the new features.

## Create matching pairs:
## Instead of creating image pairs by exhaustively searching through all possible pairs, we leverage the 
## existing colmap model and form pairs by selecting the top 20 most covisibile neighbors for each image
pairs_from_covisibility.main(
    model = colmap_model_path,
    output = sfm_pairs_path,
    num_matched = 20
)

## Use the created pairs to match images and store the matching result in a match file
match_features_conf = match_features.confs['superglue']
sfm_matches_path = match_features.main(
    conf = match_features_conf,
    pairs = sfm_pairs_path,
    features = local_feature_conf['output'], # This contains the file name where lcoal features are stored
    export_dir = output_dir
)

## Use the matches to reconstruct an SfM model
reconstruction = triangulation.main(
    sfm_dir = sfm_reconstruction_path,
    reference_model = colmap_model_path,
    image_dir = image_dir,
    pairs = sfm_pairs_path,
    features = local_features_path,
    matches = sfm_matches_path
)

In [ ]:
# Visualize the reconstruction

fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, reconstruction, color='rgba(255,0,0,0.5)', name="mapping", points_rgb=True)
fig.show()

In [ ]:
# Localize a new Image

query_image_name = 'sofa_1.png'

matcher_name = 'superglue'
match_features_conf = match_features.confs[matcher_name]
query_local_match_path = query_processing_data_dir / f'local_match_data_{matcher_name}.h5'

# Extarct local features and global descriptor for the new image
query_local_features_path = extract_features.main(
    conf = local_feature_conf,
    image_dir = queries_img_dir,
    export_dir = query_processing_data_dir,
    image_list = [query_image_name]
)

query_global_descriptor_path = extract_features.main(
    conf = global_descriptor_conf,
    image_dir = queries_img_dir,
    export_dir = query_processing_data_dir,
    image_list = [query_image_name]
)

## Use global descriptor matching to get candidate matches
nearest_candidate_images = pairs_from_retrieval.save_global_candidates_for_query(
    db_descriptors = global_descriptors_path,
    query_descriptor = query_global_descriptor_path,
    query_image_names = [query_image_name],
    num_matched = 10,
    output_file_path = query_global_matches_path
)

## Match the query image against the candidate pairs from above
match_features.match_from_paths(
    conf = match_features_conf,
    pairs_path = query_global_matches_path,
    match_path = query_local_match_path,
    feature_path_q = query_local_features_path,
    feature_path_ref = local_features_path
)

## Now we have global candidate and thier mathces. We use this, along with SfM reconstruction to localize the image.

camera = pycolmap.infer_camera_from_image(queries_img_dir / query_image_name)
ref_ids = [reconstruction.find_image_with_name(r).image_id for r in nearest_candidate_images]
conf = {
    'estimation': {'ransac': {'max_error': 12}},
    'refinement': {'refine_focal_length': True, 'refine_extra_params': True},
}
localizer = QueryLocalizer(reconstruction, conf)
ret, log = pose_from_cluster(
    localizer = localizer, 
    qname = query_image_name, 
    query_camera = camera, 
    db_ids = ref_ids, 
    features_path = local_features_path, 
    matches_path = query_local_match_path,
    features_q_path = query_local_features_path
)

print('Num_inliers: ', ret['num_inliers'])

In [ ]:
visualization.visualize_loc_from_log(
    image_dir = queries_img_dir, 
    query_name = query_image_name, 
    loc = log, 
    reconstruction = reconstruction, 
    db_image_dir = image_dir
)

In [ ]:
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, reconstruction, color='rgba(255,0,0,0.5)', points_rgb=True)
pose = pycolmap.Image(tvec=ret['tvec'], qvec=ret['qvec'])
viz_3d.plot_camera_colmap(fig, pose, camera, color='rgba(0,255,0,0.5)', name=query_image_name, fill=True)
# visualize 2D-3D correspodences
inl_3d = np.array([reconstruction.points3D[pid].xyz for pid in np.array(log['points3D_ids'])[ret['inliers']]])
viz_3d.plot_points(fig, inl_3d, color="lime", ps=1, name=query_image_name)
fig.show()